In [ ]:
# Instalando o scikit-surprise
!pip3 install scikit-surprise

In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from functools import reduce 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate, KFold

import warnings; warnings.simplefilter('ignore')

# **RECOMENDADOR BASEADO EM CONTEÚDO**

In [2]:
# Carregando o dataset movies_metadata
md_filmes = pd.read_csv('movies_metadata.csv')
# Exibindo metadados dos filmes
md_filmes

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [3]:
md_filmes['genres'] = md_filmes['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance (x, list) else [])

In [4]:
md_filmes['year'] = pd.to_datetime(md_filmes['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [5]:
md_filmes.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995


In [6]:
# Carregando dataset links_small
links_small = pd.read_csv('links_small.csv')
# Converte o tipo da coluna, filtra os não nulos e cria uma serie com os valores de tmdbId
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
# Exibindo a serie 
links_small

0          862
1         8844
2        15602
3        31357
4        11862
         ...  
9120    402672
9121    315011
9122    391698
9123    137608
9124    410803
Name: tmdbId, Length: 9112, dtype: int64

In [7]:
# Essas linhas possuem dados maus formatados, por isso foram removidas,
# o rest_index é para remover a coluna de index que foi criada, pois a 
# mesma estava fora de ordem 
md_filmes = md_filmes.drop([19730, 29503, 35587])
# Exibindo metadados dos filmes
md_filmes

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[Drama, Family]",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,NaT
45462,False,NaN,0,[Drama],NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,2011
45463,False,NaN,0,"[Action, Drama, Thriller]",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,1917


In [8]:
# Convete o tipo da coluna 
md_filmes['id'] = md_filmes['id'].astype('int')

In [9]:
# Convete o tipo da coluna popularidade para float
md_filmes['popularity'] = md_filmes['popularity'].astype('float')

In [10]:
# Para conseguir remover as duplicatas 
md_filmes['genres'] = md_filmes['genres'].astype('str')

In [11]:
# Ordenando coluna de popularidade
md_filmes = md_filmes.sort_values('popularity', ascending=False).drop_duplicates(ignore_index=True).sort_index()

In [12]:
md_filmes

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 86066, 'name': 'Despicable Me Collectio...",74000000,"['Family', 'Animation', 'Adventure', 'Comedy']",http://www.minionsmovie.com/,211672,tt2293640,en,Minions,"Minions Stuart, Kevin and Bob are recruited by...",...,1.156731e+09,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Before Gru, they had a history of bad bosses",Minions,False,6.4,4729.0,2015
1,False,"{'id': 468552, 'name': 'Wonder Woman Collectio...",149000000,"['Action', 'Adventure', 'Fantasy']",http://www.warnerbros.com/wonder-woman,297762,tt0451279,en,Wonder Woman,An Amazon princess comes to the world of Man t...,...,8.205804e+08,141.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Released,Power. Grace. Wisdom. Wonder.,Wonder Woman,False,7.2,5025.0,2017
2,False,NaN,160000000,"['Family', 'Fantasy', 'Romance']",http://movies.disney.com/beauty-and-the-beast-...,321612,tt2771200,en,Beauty and the Beast,A live-action adaptation of Disney's version o...,...,1.262886e+09,129.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Be our guest.,Beauty and the Beast,False,6.8,5530.0,2017
3,False,NaN,34000000,"['Action', 'Crime']",NaN,339403,tt3890160,en,Baby Driver,After being coerced into working for a crime b...,...,2.245113e+08,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,All you need is one killer track.,Baby Driver,False,7.2,2083.0,2017
4,False,NaN,165000000,"['Adventure', 'Family', 'Animation', 'Action',...",http://movies.disney.com/big-hero-6,177572,tt2245084,en,Big Hero 6,The special bond that develops between plus-si...,...,6.521054e+08,102.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,From the creators of Wreck-it Ralph and Frozen,Big Hero 6,False,7.8,6289.0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45441,False,NaN,0,[],NaN,257608,tt0319685,en,Khanei ruye ab,Director Bahman Farmanara's second film follow...,...,0.000000e+00,111.0,[],Released,The midlife crisis of a successful professiona...,Khanei ruye ab,False,0.0,0.0,NaT
45442,False,NaN,0,['Comedy'],NaN,92292,tt0067787,en,Star Spangled Girl,I may be provincial and old-fashioned. I may b...,...,0.000000e+00,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Star Spangled Girl,False,0.0,0.0,1971
45443,False,NaN,0,"['Action', 'Thriller', 'Drama']",NaN,82663,tt0113002,en,Midnight Man,British soldiers force a recently captured IRA...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
45444,False,"{'id': 122661, 'name': 'Mardock Scramble Colle...",0,"['Animation', 'Science Fiction']",http://m-scramble.jp/exhaust/,122662,tt2423504,ja,マルドゥック・スクランブル 排気,Third film of the Mardock Scramble series.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [13]:
# Removendo os duplicados que contém mesmo valore de id, imdb_id, title
md_filmes = md_filmes.drop_duplicates(subset=['id', 'imdb_id', 'title'], ignore_index=True)

In [14]:
# Convete o tipo da coluna genero para list
md_filmes['genres'] = md_filmes['genres'].apply(literal_eval)

### ***RECOMENDADOR BASEADO EM METADADOS DOS FILMES***

In [15]:
# Carregando dados dos credits  
credits = pd.read_csv('credits.csv')
# Carregando dados dos keywords  
keywords = pd.read_csv('keywords.csv')

In [16]:
# Converte o tipo da coluna para int
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md_filmes['id'] = md_filmes['id'].astype('int')

In [17]:
md_filmes.shape

(45433, 25)

In [18]:
# Fazendo a junção dos Dataframes de filmes com creditos e palavras chave 
md_filmes = md_filmes.merge(credits, on='id')
md_filmes = md_filmes.merge(keywords, on='id')

In [19]:
# Para conseguir remover as duplicatas 
md_filmes['genres'] = md_filmes['genres'].astype('str')

In [20]:
md_filmes = md_filmes.drop_duplicates()

In [21]:
# Convete o tipo da coluna genero para list
md_filmes['genres'] = md_filmes['genres'].apply(literal_eval)

In [22]:
# Colete todos os metadados para filmes no conjunto de dados links_small
# Filtre os filmes cujo id está presente no arquivo links.csv e armazene em smd
# .isin -> Se cada elemento no DataFrame está contido em valores.
# Similar movie data - smd
smd = md_filmes[md_filmes['id'].isin(links_small)]
smd.shape

(9086, 28)

In [23]:
smd

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords
0,False,"{'id': 86066, 'name': 'Despicable Me Collectio...",74000000,"[Family, Animation, Adventure, Comedy]",http://www.minionsmovie.com/,211672,tt2293640,en,Minions,"Minions Stuart, Kevin and Bob are recruited by...",...,Released,"Before Gru, they had a history of bad bosses",Minions,False,6.4,4729.0,2015,"[{'cast_id': 22, 'character': 'Scarlet Overkil...","[{'credit_id': '5431b2b10e0a2656e20026c7', 'de...","[{'id': 3487, 'name': 'assistant'}, {'id': 179..."
4,False,NaN,165000000,"[Adventure, Family, Animation, Action, Comedy]",http://movies.disney.com/big-hero-6,177572,tt2245084,en,Big Hero 6,The special bond that develops between plus-si...,...,Released,From the creators of Wreck-it Ralph and Frozen,Big Hero 6,False,7.8,6289.0,2014,"[{'cast_id': 23, 'character': 'Baymax (voice)'...","[{'credit_id': '537534e7c3a3681efb0002b3', 'de...","[{'id': 380, 'name': 'brother brother relation..."
5,False,"{'id': 448150, 'name': 'Deadpool Collection', ...",58000000,"[Action, Adventure, Comedy]",http://www.foxmovies.com/movies/deadpool,293660,tt1431045,en,Deadpool,Deadpool tells the origin story of former Spec...,...,Released,Witness the beginning of a happy ending,Deadpool,False,7.4,11444.0,2016,"[{'cast_id': 99, 'character': 'Wade Wilson / D...","[{'credit_id': '57e564fac3a3681beb0000cc', 'de...","[{'id': 2095, 'name': 'anti hero'}, {'id': 307..."
7,False,"{'id': 87096, 'name': 'Avatar Collection', 'po...",237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",...,Released,Enter the World of Pandora.,Avatar,False,7.2,12114.0,2009,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."
8,False,"{'id': 404609, 'name': 'John Wick Collection',...",20000000,"[Action, Thriller]",http://www.johnwickthemovie.com/,245891,tt2911666,en,John Wick,Ex-lunatic John Wick comes off his meds to tra...,...,Released,Don't set him off.,John Wick,False,7.0,5499.0,2014,"[{'cast_id': 6, 'character': 'John Wick', 'cre...","[{'credit_id': '52fe4f0cc3a36847f82b9c2b', 'de...","[{'id': 2708, 'name': 'hitman'}, {'id': 8045, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46065,False,NaN,0,"[Drama, Foreign]",NaN,12632,tt0110246,de,Kaspar Hauser,No overview found.,...,Released,NaN,Kaspar Hauser,False,6.3,4.0,1993,"[{'cast_id': 1, 'character': 'Kaspar Hauser', ...","[{'credit_id': '52fe450c9251416c75049e4b', 'de...","[{'id': 1563, 'name': 'prisoner'}, {'id': 2279..."
46130,False,NaN,0,[],NaN,124306,tt0106810,en,En compagnie d'Antonin Artaud,"May, 1946, in Paris young poet Jacques Prevel ...",...,Released,NaN,My Life and Times With Antonin Artaud,False,8.0,1.0,1993,"[{'cast_id': 2, 'character': 'Antonin Artaud',...","[{'credit_id': '52fe4aa8c3a368484e160327', 'de...",[]
46200,False,NaN,0,"[Documentary, Foreign]",NaN,62437,tt0067324,de,Land des Schweigens und der Dunkelheit,"Through examining Fini Straubinger, an old wom...",...,Released,NaN,Land of Silence and Darkness,False,7.5,16.0,1971,"[{'cast_id': 3, 'character': '', 'credit_id': ...","[{'credit_id': '52fe467dc3a368484e093927', 'de...",[]
46238,False,NaN,0,"[TV Movie, Drama, Romance, Comedy]",NaN,53185,tt0111709,en,Wide Eyed and Legless,Diana and Deric have an ideal marriage: they t...,...,Released,NaN,Wide Eyed and Legless,False,0.0,0.0,1993,"[{'cast_id': 1, 'character': 'Diana Longden', ...","[{'credit_id': '58a79904c3a36876e8000dde', 'de...",[]


In [24]:
smd['cast'] = smd['cast'].apply(literal_eval)

In [25]:
smd['crew'] = smd['crew'].apply(literal_eval)

In [26]:
smd['keywords'] = smd['keywords'].apply(literal_eval)

In [27]:
# Cria nova coluna com tamanho do cast
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))

In [28]:
# Cria nova coluna com tamanho do crew
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [29]:
smd.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,title,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size
0,False,"{'id': 86066, 'name': 'Despicable Me Collectio...",74000000,"[Family, Animation, Adventure, Comedy]",http://www.minionsmovie.com/,211672,tt2293640,en,Minions,"Minions Stuart, Kevin and Bob are recruited by...",...,Minions,False,6.4,4729.0,2015,"[{'cast_id': 22, 'character': 'Scarlet Overkil...","[{'credit_id': '5431b2b10e0a2656e20026c7', 'de...","[{'id': 3487, 'name': 'assistant'}, {'id': 179...",17,12
4,False,NaN,165000000,"[Adventure, Family, Animation, Action, Comedy]",http://movies.disney.com/big-hero-6,177572,tt2245084,en,Big Hero 6,The special bond that develops between plus-si...,...,Big Hero 6,False,7.8,6289.0,2014,"[{'cast_id': 23, 'character': 'Baymax (voice)'...","[{'credit_id': '537534e7c3a3681efb0002b3', 'de...","[{'id': 380, 'name': 'brother brother relation...",46,39
5,False,"{'id': 448150, 'name': 'Deadpool Collection', ...",58000000,"[Action, Adventure, Comedy]",http://www.foxmovies.com/movies/deadpool,293660,tt1431045,en,Deadpool,Deadpool tells the origin story of former Spec...,...,Deadpool,False,7.4,11444.0,2016,"[{'cast_id': 99, 'character': 'Wade Wilson / D...","[{'credit_id': '57e564fac3a3681beb0000cc', 'de...","[{'id': 2095, 'name': 'anti hero'}, {'id': 307...",46,88
7,False,"{'id': 87096, 'name': 'Avatar Collection', 'po...",237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",...,Avatar,False,7.2,12114.0,2009,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de...","[{'id': 1463, 'name': 'culture clash'}, {'id':...",83,153
8,False,"{'id': 404609, 'name': 'John Wick Collection',...",20000000,"[Action, Thriller]",http://www.johnwickthemovie.com/,245891,tt2911666,en,John Wick,Ex-lunatic John Wick comes off his meds to tra...,...,John Wick,False,7.0,5499.0,2014,"[{'cast_id': 6, 'character': 'John Wick', 'cre...","[{'credit_id': '52fe4f0cc3a36847f82b9c2b', 'de...","[{'id': 2708, 'name': 'hitman'}, {'id': 8045, ...",35,57


In [30]:
def obter_diretor(x):
  for i in x:
    if i['job'] == 'Director':
      return i['name']
  return np.nan

In [31]:
smd['director'] = smd['crew'].apply(obter_diretor)

In [32]:
# Organiza o nome do cast
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [33]:
# Seleciona os 3 principais atores 
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [34]:
smd.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director
0,False,"{'id': 86066, 'name': 'Despicable Me Collectio...",74000000,"[Family, Animation, Adventure, Comedy]",http://www.minionsmovie.com/,211672,tt2293640,en,Minions,"Minions Stuart, Kevin and Bob are recruited by...",...,False,6.4,4729.0,2015,"[Sandra Bullock, Jon Hamm, Michael Keaton]","[{'credit_id': '5431b2b10e0a2656e20026c7', 'de...","[{'id': 3487, 'name': 'assistant'}, {'id': 179...",17,12,Kyle Balda
4,False,NaN,165000000,"[Adventure, Family, Animation, Action, Comedy]",http://movies.disney.com/big-hero-6,177572,tt2245084,en,Big Hero 6,The special bond that develops between plus-si...,...,False,7.8,6289.0,2014,"[Scott Adsit, Ryan Potter, Daniel Henney]","[{'credit_id': '537534e7c3a3681efb0002b3', 'de...","[{'id': 380, 'name': 'brother brother relation...",46,39,Chris Williams
5,False,"{'id': 448150, 'name': 'Deadpool Collection', ...",58000000,"[Action, Adventure, Comedy]",http://www.foxmovies.com/movies/deadpool,293660,tt1431045,en,Deadpool,Deadpool tells the origin story of former Spec...,...,False,7.4,11444.0,2016,"[Ryan Reynolds, Morena Baccarin, Ed Skrein]","[{'credit_id': '57e564fac3a3681beb0000cc', 'de...","[{'id': 2095, 'name': 'anti hero'}, {'id': 307...",46,88,Tim Miller
7,False,"{'id': 87096, 'name': 'Avatar Collection', 'po...",237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",...,False,7.2,12114.0,2009,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{'credit_id': '52fe48009251416c750aca23', 'de...","[{'id': 1463, 'name': 'culture clash'}, {'id':...",83,153,James Cameron
8,False,"{'id': 404609, 'name': 'John Wick Collection',...",20000000,"[Action, Thriller]",http://www.johnwickthemovie.com/,245891,tt2911666,en,John Wick,Ex-lunatic John Wick comes off his meds to tra...,...,False,7.0,5499.0,2014,"[Keanu Reeves, Michael Nyqvist, Alfie Allen]","[{'credit_id': '52fe4f0cc3a36847f82b9c2b', 'de...","[{'id': 2708, 'name': 'hitman'}, {'id': 8045, ...",35,57,Chad Stahelski


In [37]:
smd['keywords'][0]

[{'id': 3487, 'name': 'assistant'},
 {'id': 179430, 'name': 'aftercreditsstinger'},
 {'id': 179431, 'name': 'duringcreditsstinger'},
 {'id': 189481, 'name': 'evil mastermind'},
 {'id': 190999, 'name': 'minions'},
 {'id': 209714, 'name': '3d'}]

In [38]:
# Organiza as palavras chaves numa lista
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else []) 

In [39]:
# Remove os espaços e deixa tudo minusculo no cast
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [40]:
# Converte o tipo da coluna do director para string, pega o nome do diretor deixa minusculo e sem espaços
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))

In [41]:
# Repete 3 vezes o nome do diretor na lista para dar mais peso em relação a todo elenco
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

#### **PALAVRAS-CHAVE**  
Será feito um pré-processamento das palavras chaves

In [42]:
smd

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director
0,False,"{'id': 86066, 'name': 'Despicable Me Collectio...",74000000,"[Family, Animation, Adventure, Comedy]",http://www.minionsmovie.com/,211672,tt2293640,en,Minions,"Minions Stuart, Kevin and Bob are recruited by...",...,False,6.4,4729.0,2015,"[sandrabullock, jonhamm, michaelkeaton]","[{'credit_id': '5431b2b10e0a2656e20026c7', 'de...","[assistant, aftercreditsstinger, duringcredits...",17,12,"[kylebalda, kylebalda, kylebalda]"
4,False,NaN,165000000,"[Adventure, Family, Animation, Action, Comedy]",http://movies.disney.com/big-hero-6,177572,tt2245084,en,Big Hero 6,The special bond that develops between plus-si...,...,False,7.8,6289.0,2014,"[scottadsit, ryanpotter, danielhenney]","[{'credit_id': '537534e7c3a3681efb0002b3', 'de...","[brother brother relationship, hero, talent, r...",46,39,"[chriswilliams, chriswilliams, chriswilliams]"
5,False,"{'id': 448150, 'name': 'Deadpool Collection', ...",58000000,"[Action, Adventure, Comedy]",http://www.foxmovies.com/movies/deadpool,293660,tt1431045,en,Deadpool,Deadpool tells the origin story of former Spec...,...,False,7.4,11444.0,2016,"[ryanreynolds, morenabaccarin, edskrein]","[{'credit_id': '57e564fac3a3681beb0000cc', 'de...","[anti hero, mercenary, marvel comic, superhero...",46,88,"[timmiller, timmiller, timmiller]"
7,False,"{'id': 87096, 'name': 'Avatar Collection', 'po...",237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",...,False,7.2,12114.0,2009,"[samworthington, zoesaldana, sigourneyweaver]","[{'credit_id': '52fe48009251416c750aca23', 'de...","[culture clash, future, space war, space colon...",83,153,"[jamescameron, jamescameron, jamescameron]"
8,False,"{'id': 404609, 'name': 'John Wick Collection',...",20000000,"[Action, Thriller]",http://www.johnwickthemovie.com/,245891,tt2911666,en,John Wick,Ex-lunatic John Wick comes off his meds to tra...,...,False,7.0,5499.0,2014,"[keanureeves, michaelnyqvist, alfieallen]","[{'credit_id': '52fe4f0cc3a36847f82b9c2b', 'de...","[hitman, russian mafia, revenge, murder, gangs...",35,57,"[chadstahelski, chadstahelski, chadstahelski]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46065,False,NaN,0,"[Drama, Foreign]",NaN,12632,tt0110246,de,Kaspar Hauser,No overview found.,...,False,6.3,4.0,1993,"[andréeisermann, udosamel, jeremyclyde]","[{'credit_id': '52fe450c9251416c75049e4b', 'de...","[prisoner, bavaria, nuremberg, mensch als obje...",9,8,"[petersehr, petersehr, petersehr]"
46130,False,NaN,0,[],NaN,124306,tt0106810,en,En compagnie d'Antonin Artaud,"May, 1946, in Paris young poet Jacques Prevel ...",...,False,8.0,1.0,1993,[samifrey],"[{'credit_id': '52fe4aa8c3a368484e160327', 'de...",[],1,1,"[gérardmordillat, gérardmordillat, gérardmordi..."
46200,False,NaN,0,"[Documentary, Foreign]",NaN,62437,tt0067324,de,Land des Schweigens und der Dunkelheit,"Through examining Fini Straubinger, an old wom...",...,False,7.5,16.0,1971,"[finistraubinger, heinrichfleischmann, vladimi...","[{'credit_id': '52fe467dc3a368484e093927', 'de...",[],7,6,"[wernerherzog, wernerherzog, wernerherzog]"
46238,False,NaN,0,"[TV Movie, Drama, Romance, Comedy]",NaN,53185,tt0111709,en,Wide Eyed and Legless,Diana and Deric have an ideal marriage: they t...,...,False,0.0,0.0,1993,"[juliewalters, jimbroadbent, thorahird]","[{'credit_id': '58a79904c3a36876e8000dde', 'de...",[],32,8,"[richardloncraine, richardloncraine, richardlo..."


In [43]:
# Criar uma serie palavras chave aplicando ao longo do axis=1 coluna
# stack ->retornar um dataframe remodelado, se as colunas tiverem um único nível, a saída é uma Série;
# Reseta index em 1 nível
palavras_chave = smd.apply(lambda x: pd.Series(x['keywords']), axis=1).stack().reset_index(level=1, drop=True)

In [44]:
# Nomeia a serie
palavras_chave.name = 'keyword'

In [45]:
# Retorna uma série contendo contagens de valores exclusivos.
palavras_chave = palavras_chave.value_counts()

#Exibindo as 5 palavras que ocorrem com mais frequencia  
palavras_chave[:5]

independent film        603
woman director          541
murder                  397
duringcreditsstinger    327
based on novel          310
Name: keyword, dtype: int64

In [46]:
# Selecionando somente as palavras que ocorrem mais de uma vez
palavras_chave = palavras_chave[palavras_chave > 1]

In [47]:
# Converterndo palavras em seu radical
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [48]:
def filtrar_palavras_chave(x):
  words = []
  for i in x:
    if i in palavras_chave:
      words.append(i)
  return words

In [49]:
# Aplicando a função filtrar palavras chaves
smd['keywords'] = smd['keywords'].apply(filtrar_palavras_chave)

In [50]:
# Aplica a lisa de palavras o stemmer que deixa somente o radical
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])

In [51]:
# Deixa as palavras minusculas e remove os espaços em palavras compostas 
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [52]:
# Criando sopa de palavras unindo as colunas 
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']

In [53]:
# Une as palavras da lista separando por espaços
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [54]:
# Ele é usado para transformar um determinado texto em um vetor com base na frequência (contagem) 
# de cada palavra que ocorre em todo o texto.
count = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
# fit_transform -> Aprenda o dicionário de vocabulário e retorne a matriz documento-termo.
#       Isso é equivalente a fit seguido de transformação, mas implementado de forma mais eficiente.
count_matrix = count.fit_transform(smd['soup'])

In [55]:
# Calcule a similaridade de cosseno entre as amostras em X e Y.
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [56]:
# Cria uma coluna index contendo os indices do df - reset_index() -> Redefina o índice ou um nível dele.
smd = smd.reset_index()
# Cria uma serie com os titulos 
titles = smd['title']
# Faço o mapa reverso onde o id é o titulo e os valores são o id do filme
# Series -> Array unidimensional com rótulos de eixo (incluindo séries temporais).
indices = pd.Series(smd.index, index=smd['title'])

In [57]:
indices

title
Minions                                     0
Big Hero 6                                  1
Deadpool                                    2
Avatar                                      3
John Wick                                   4
                                         ... 
Kaspar Hauser                            9081
My Life and Times With Antonin Artaud    9082
Land of Silence and Darkness             9083
Wide Eyed and Legless                    9084
All the Vermeers in New York             9085
Length: 9086, dtype: int64

#**FILTRAGEM COLABORATIVA**

In [58]:
reader = Reader()

In [59]:
avaliacoes = pd.read_csv('ratings_small.csv')
avaliacoes.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [60]:
dados = Dataset.load_from_df(avaliacoes[['userId', 'movieId', 'rating']], reader)

In [61]:
svd = SVD()
cross_validate(svd, dados, measures=['RMSE', 'MAE'], verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9002  0.8999  0.8939  0.8991  0.8868  0.8960  0.0052  
MAE (testset)     0.6932  0.6910  0.6878  0.6931  0.6834  0.6897  0.0037  
Fit time          1.91    1.75    1.95    1.47    1.45    1.71    0.21    
Test time         0.25    0.41    0.20    0.17    0.33    0.27    0.09    


{'test_rmse': array([0.90021693, 0.89994404, 0.89386288, 0.89914217, 0.88675215]),
 'test_mae': array([0.69318794, 0.69101391, 0.68783834, 0.69305169, 0.68339881]),
 'fit_time': (1.9117357730865479,
  1.751526117324829,
  1.9486916065216064,
  1.4727180004119873,
  1.4537498950958252),
 'test_time': (0.24646210670471191,
  0.4105963706970215,
  0.20069265365600586,
  0.17473626136779785,
  0.3254091739654541)}

In [62]:
trainset = dados.build_full_trainset()
svd.fit(trainset)

In [63]:
avaliacoes[avaliacoes['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [64]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.7171519606386743, details={'was_impossible': False})

# **FUNÇÕES UTILITARIAS**

In [65]:
# Método que normaliza valores entre 0 e 1
# Recebe uma coluna como serie
def normalizacao_valores(serie):
  serie_index = serie.index
  array = serie.values
  y = array.reshape(-1, 1)
  scaler = MinMaxScaler(feature_range= (0, 1))
  rescaled = scaler.fit_transform(y)
  array = rescaled
  serie_normalizada = pd.DataFrame(array, index=serie_index)
  return serie_normalizada

In [66]:
# Aqui as vezes retorna um int e as vezes retorna uma serie tratei dessa forma
def get_index(title):
  # Df indices contem o titulo e o código do filme ao lado
  idx = indices[title]

  if isinstance(idx, pd.Series):
    print('Entrou')
    return idx[0]

  return idx

# **RECOMENDADOR KNN**  

Antes de qualquer coisa, ajustar o Dataset e organizar as colunas que irei utilizar o dataset de exemplo que irei me basear é esse:   
(https://github.com/krishnaik06/Recommendation_complete_tutorial/tree/master/KNN%20Movie%20Recommendation)  
Assim que estiver em Pedrinhas tenho que deixar o dataset que tenho conforme será utilizado aqui, realizar o pre-processado com os dados do meu Dataset.  
***Esse algoritmo utiliza a estratégia de filtragem colaborativa***  
Existem dois tipos:
* Baseado em itens;   
* Baseado em Usuários.  

***No nosso caso iremos utilizar a filtragem colaborativa baseado em usuários***

In [67]:
def recomendacao_KNN(usuarioId):

  # Importando dataset
  ds_filmes = pd.read_csv('movies_metadata.csv', dtype={'imdb_id':"string"})
  ds_links = pd.read_csv('links_small.csv', dtype={'movieId':"Int64",'imdbId':"Int64",'tmdbId':"Int64"})
  ds_avaliacoes = pd.read_csv('ratings_small.csv')

  # Cria serie com tmdbId não nulos de DS_Links
  serie_tmdbid_ds_links = ds_links[ds_links['tmdbId'].notnull()]['tmdbId'].astype('int')

  # Remoção de linhas mal formatadas 
  ds_filmes = ds_filmes.drop([19730, 29503, 35587])

  # Converte o tipo da coluna
  ds_filmes['id'] = ds_filmes['id'].astype('int')

  # Criando serie com filmes filtrando por serie_tmdbid_ds_links
  ds_filmes = ds_filmes[ds_filmes['id'].isin(serie_tmdbid_ds_links)]

  # Verificando imdbids nulos
  ds_filmes[ds_filmes['imdb_id'].isnull()]

  # Verificando ids nulos
  ds_filmes[ds_filmes['id'].isnull()]


  # Extraindo coluna imdbId para pegar o código do filme em links
  ds_filmes['imdbId'] = ds_filmes['imdb_id'].apply(lambda x: x if pd.isna(x) else str(x)[2:])

  # Alterando tipo da coluna
  ds_filmes['imdbId'] = ds_filmes['imdbId'].astype('int')

  # Merge pegando os movieIds de Links
  ds_filmes = pd.merge(ds_filmes, ds_links, on='imdbId')

  # Removendo a coluna tmdbId
  ds_filmes.drop(['tmdbId'], axis=1, inplace=True)
  
  # Para fazer a filtragem colaborativa o que me interessa é o Id do Filme e o titulo
  # Selecionando/filtrandos as colunas que irei utilizar 
  ds_filmes = ds_filmes[['movieId', 'title']]

  # Filtrando as colunas que preciso
  ds_avaliacoes = ds_avaliacoes[['userId', 'movieId', 'rating']]
  
  # Juntar as duas bases, semelhante ao join no SQL 
  df_filmes_avaliacoes = pd.merge(ds_filmes, ds_avaliacoes, on='movieId')
  

  # Pega o id do usuário e coloca como indice da coluna e pivotar a tabela 
  # Onde os titulos apareçam em cima e as avaliações serão os valores 
  df_recommender = df_filmes_avaliacoes.pivot_table(index='userId', columns='title', values='rating').fillna(0)
  

  # Modelo dos vizinhos próximos - KNN
  # Distância do cosseno não sofre muito com a maldição da dimensionalidade
  # Instancia o modelo KNN, usando a metrica do cosseno(hiperparametros)
  modelo_knn = NearestNeighbors(metric='cosine')
  # Calcula todas as distâncias em relação aos vizinhos próximos
  modelo_knn.fit(df_recommender)

  # Quantidade de vizinhos mais próximos que quero
  qtde_vizinhos = 4

  # Recupera a linha que o usuário se encontra
  # Usuario que eu quero recomendar algo
  idx_usuario_knn = df_recommender.index.get_loc(df_recommender.loc[usuarioId].name)
  # Quando chamar o modelo de KNN preciso saber a distância dos vizinhos próximos
  # E quais são os indices vizinhos próximos
  # Irei passar no parâmetro de kneighbors() o registro que quero usar para puxar
  # Vizinho mais próximo - Fiz uma mudança ao invés de iloc usei loc
  distancia_vizinhos, indices_vizinhos = modelo_knn.kneighbors(df_recommender.iloc[idx_usuario_knn].values.reshape(1, -1), n_neighbors=qtde_vizinhos)

  usuario = df_recommender.index[idx_usuario_knn]

  # Cria a lista que irá conter os dataframes
  dfs_vizinhos_proximo = []

  # For para percorrer cada um da lista 
  # Usei o flatter para remover uma dimensão 
  for i in range(0, len(distancia_vizinhos.flatten())):
    # Pula o primeiro elemento pois vai ser ele mesmo
    if i == 0:
      ds_usuario_knn = df_recommender.loc[usuario].to_frame()
      dfs_vizinhos_proximo.append(ds_usuario_knn)
    else:
      # Localiza o vizinho mais próximo em df_recommender
      vizinho_proximo = df_recommender.index[indices_vizinhos.flatten()[i]]
      # Adiciona o vizinho próximo a lista dos dataframes
      dfs_vizinhos_proximo.append(df_recommender.loc[vizinho_proximo].to_frame())
  

  # Pegando o indice dos vizinhos mais próximos
  idx_vizinhos_prox = [col for i in range(len(dfs_vizinhos_proximo)) for col in dfs_vizinhos_proximo[i] if i != 0]
  
  # Faz o merge da dataframe do usuário com seus vizinhos
  ds_titulos = reduce(lambda df_esq, df_dir: pd.merge(df_esq, df_dir, on=['title']), dfs_vizinhos_proximo)

  # Ordena de forma descrescente pelos vizinhos mais próximos
  ds_titulos = ds_titulos.sort_values(by=idx_vizinhos_prox, ascending=False)
  
  # Filtrar todos os titulos do vizinho que foram > 0 e onde usuário não assistiu
  ds_titulos = ds_titulos[((ds_titulos[idx_vizinhos_prox[0]] > 0) | (ds_titulos[idx_vizinhos_prox[1]] > 0) | (ds_titulos[idx_vizinhos_prox[2]] > 0)) & (ds_titulos[usuario] == 0)]
  
  # Criando e calculando a média 
  ds_titulos['Media'] = (ds_titulos[idx_vizinhos_prox[0]] + ds_titulos[idx_vizinhos_prox[1]] + ds_titulos[idx_vizinhos_prox[2]]) / len(idx_vizinhos_prox)

  # Resetando o indice
  ds_titulos.reset_index(inplace=True)

  # Atribui a média os valores de média normalizados 
  ds_titulos['est'] = normalizacao_valores(ds_titulos['Media'])

  # Filtrando as colunas que irei devolver ao método híbrido 
  titulos_recomendacao_KNN = ds_titulos[['title', 'est']]
  titulos_recomendacao_KNN = titulos_recomendacao_KNN.sort_values(by=['est'], ascending=False)


  return titulos_recomendacao_KNN.head(10)


**Testando Recomendador KNN**

In [68]:
recomendacao_KNN(1)

,title,est
0,Beverly Hills Cop,1.000000
13,The Godfather,0.642857
30,Titus,0.642857
1,The Brady Bunch Movie,0.642857
15,The Shawshank Redemption,0.642857
28,Proof,0.642857
14,The Godfather: Part II,0.642857
29,Three Kings,0.642857
31,A Perfect Murder,0.571429
32,Sylvia,0.571429


#**RECOMENDADOR HÍBRIDO**

In [69]:
def convert_int(x):
  try:
    return int(x)
  except:
    return np.nan

In [70]:
# Carregando o arquivo que contem os ids dos filmes, selecionando 
# as colunas 'movieId', 'tmdbId'
id_map = pd.read_csv('links_small.csv')[['movieId', 'tmdbId']]
# Converte a coluna id_map['tmdbId'] para inteiro
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
# Mudando os nomes da coluna tmdbId para id
id_map.columns = ['movieId', 'id']
# Fazendo o merge de id_map com SMD
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

In [71]:
indices_map = id_map.set_index('id')

In [72]:
# Junta e soma as estimativas de cada dataframe 
def agrupa_recomendacoes(dataframe1, dataframe2):
  dfs_concatenados = pd.concat([dataframe1, dataframe2], ignore_index=True)
  dfs_agrupados = dfs_concatenados.groupby('title')[['est']].mean()
  # dfs_agrupados = dfs_concatenados.groupby('title')['est'].sum()

  dfs_agrupados = dfs_agrupados.sort_values('est', ascending=False)
  dfs_agrupados = dfs_agrupados.reset_index()

  return dfs_agrupados


In [73]:
def recomendador_hibrido(userId, title):

  if isinstance(userId, int) and isinstance(title, str):

    if (userId in avaliacoes['userId'].values) and (title in id_map.index) :
      
      idx = get_index(title)

      titulos_recomendados_KNN = recomendacao_KNN(userId)
  
      sim_scores = list(enumerate(cosine_sim[int(idx)]))
      sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
      sim_scores = sim_scores[1:26]
      movie_indices = [i[0] for i in sim_scores]

      movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
      movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
      movies['est'] = normalizacao_valores(movies['est'])
      movies = movies[['title', 'est']]

      movies = movies.sort_values('est', ascending=False)

      movies = movies.head(10)

      recomendacao_hibrida = agrupa_recomendacoes(titulos_recomendados_KNN, movies)

      return recomendacao_hibrida
    
    else:
      print('Erro: O id do usuário ou nome do filme não consta no dataset')
  else:
    print('Algo inesperado aconteceu. Tente novamente!')

In [81]:
recomendador_hibrido(1, "Avatar")

,title,est
0,Beverly Hills Cop,1.000000
1,Aliens,1.000000
2,Star Trek Into Darkness,0.995495
3,X-Men: Days of Future Past,0.976158
4,Terminator 2: Judgment Day,0.950019
5,Star Wars: The Force Awakens,0.941489
6,The Terminator,0.916481
7,The Abyss,0.773472
8,Darby O'Gill and the Little People,0.767317
9,True Lies,0.748287


In [82]:
recomendador_hibrido(500, 'Avatar')

,title,est
0,Walk the Line,1.000000
1,Star Trek Into Darkness,1.000000
2,"I, Robot",0.920000
3,Apollo 13,0.920000
4,X-Men: Days of Future Past,0.904441
5,Spider-Man,0.880000
6,Rain Man,0.880000
7,Being John Malkovich,0.880000
8,Titanic,0.861675
9,Million Dollar Baby,0.840000
